# Answering Business Questions using SQL

<b>Data Model</b>

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

<b>Sample Data</b>

- Media related data was created using real data from an iTunes Library. It is possible for you to use your own iTunes Library to generate the SQL scripts, see instructions here.

- Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.).

- Sales information is auto generated using random data for a four year period.

## Initial Setup 

In [3]:
# import modules
import sqlite3
import pandas as pd

# return pandas dataframe of a sql query
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)
    
# executes an sql command using the sqlite module
def run_command(command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(command)
        
# calls the run_query() to return a list of all tables and views in the db
def show_tables():
    q = "SELECT name, type FROM sqlite_master WHERE type IN (\"table\", \"view\")"
    result = run_query(q)
    print(result)
    
# display the tables and views in the db
show_tables()

              name   type
0            album  table
1           artist  table
2         customer  table
3         employee  table
4            genre  table
5          invoice  table
6     invoice_line  table
7       media_type  table
8         playlist  table
9   playlist_track  table
10           track  table


## Selecting the Albums to Purchase 

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|      Artist Name     |  Genre  |
|:--------------------:|:-------:|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [7]:
print(run_query("SELECT * FROM invoice;"))

     invoice_id  customer_id         invoice_date  \
0             1           18  2017-01-03 00:00:00   
1             2           30  2017-01-03 00:00:00   
2             3           40  2017-01-05 00:00:00   
3             4           18  2017-01-06 00:00:00   
4             5           27  2017-01-07 00:00:00   
5             6           31  2017-01-10 00:00:00   
6             7           49  2017-01-12 00:00:00   
7             8           59  2017-01-13 00:00:00   
8             9           18  2017-01-18 00:00:00   
9            10           31  2017-01-18 00:00:00   
10           11           38  2017-01-20 00:00:00   
11           12           42  2017-01-21 00:00:00   
12           13           35  2017-01-22 00:00:00   
13           14           25  2017-01-23 00:00:00   
14           15           44  2017-01-25 00:00:00   
15           16            1  2017-01-26 00:00:00   
16           17           20  2017-01-28 00:00:00   
17           18           24  2017-02-02 00:00